In [3]:
# imports for the practice (you can add more if you need)
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split
import shutil


# %matplotlib notebook
%matplotlib inline

import xml.etree.ElementTree as ET

seed = 212
np.random.seed(seed)
torch.manual_seed(seed)

from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/046211_Deep_Learning/DL_project'



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/046211_Deep_Learning/DL_project


# Sources
⭐1. training yolo v5 on a custom dataset: https://blog.paperspace.com/train-yolov5-custom-data/
2. pytorch data loading tutorial: https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/21adbaecd47a412f8143afb1c48f05a6/data_loading_tutorial.ipynb#scrollTo=6S5I0VfvBsPq
3. pytorch transfer learning for CV: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html?highlight=transfer_learning_tutorial


# Load model, freeze layers
----

In [4]:
# model = torchvision.models.detection.mask_rcnn(pretrained=False, progress=True, num_classes=91, pretrained_backbone=True, trainable_backbone_layers=None, **kwargs)
# model = torchvision.models.googlenet(pretrained = True, progress=True)
# model.eval()

# YOLO V5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-1-7 torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [5]:
model.eval()

# show model layers
# for k, v in model.named_parameters():
#     print(k)

# Freeze 
freeze = 23
freeze = [f'model.{x}.' for x in range(freeze)]  # layers to freeze 
for k, v in model.named_parameters(): 
    v.requires_grad = True  # train all layers 
    if any(x in k for x in freeze): 
        print(f'freezing {k}') 
        v.requires_grad = False 
    else:
      print(f'not freezing {k}') 

freezing model.model.model.0.conv.weight
freezing model.model.model.0.conv.bias
freezing model.model.model.1.conv.weight
freezing model.model.model.1.conv.bias
freezing model.model.model.2.cv1.conv.weight
freezing model.model.model.2.cv1.conv.bias
freezing model.model.model.2.cv2.conv.weight
freezing model.model.model.2.cv2.conv.bias
freezing model.model.model.2.cv3.conv.weight
freezing model.model.model.2.cv3.conv.bias
freezing model.model.model.2.m.0.cv1.conv.weight
freezing model.model.model.2.m.0.cv1.conv.bias
freezing model.model.model.2.m.0.cv2.conv.weight
freezing model.model.model.2.m.0.cv2.conv.bias
freezing model.model.model.3.conv.weight
freezing model.model.model.3.conv.bias
freezing model.model.model.4.cv1.conv.weight
freezing model.model.model.4.cv1.conv.bias
freezing model.model.model.4.cv2.conv.weight
freezing model.model.model.4.cv2.conv.bias
freezing model.model.model.4.cv3.conv.weight
freezing model.model.model.4.cv3.conv.bias
freezing model.model.model.4.m.0.cv1.con

# Dataset helper functions
extract data from xml, convert to YOLOv5 format


In [6]:
# Function to get the data from XML Annotation

def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    
    # Initialise the info dict 
    info_dict = {}
    info_dict['bboxes'] = []

    # Parse the XML Tree
    for elem in root:
        # Get the file name 
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
            
        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)
        
        # Get details of the bounding box 
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    
    return info_dict

In [7]:

# MANUAL EXTRACTION OF INFO FROM XML
#-----------------------------------
# tree = ET.parse('/content/drive/MyDrive/Colab Notebooks/046211_Deep_Learning/DL_project/data/Knife_detection/xmls/armas (1).xml')
# root = tree.getroot()
## name:
# print(root[6][0].text)
## coordinates: xmin
# print(root[6][4][0].text)
# ymin - [6][4][1]
# xmax - [6][4][2]
# yman - [6][4][3]
# for obj in root.iter('object'):
#   print(obj[0].text)
#   print(obj[4][0].text)
#   print(obj[4][1].text)
#   print(obj[4][2].text)
#   print(obj[4][3].text)
#-----------------------------------
# MANUAL EXTRACTION OF INFO FROM XML



# USING THE ABOVE FUNCTION
#-------------------------
# example = extract_info_from_xml('/content/drive/MyDrive/Colab Notebooks/046211_Deep_Learning/DL_project/data/Knife_detection/xmls/armas (1).xml')
# print(example)

In [8]:
# Dictionary that maps class names to IDs
class_name_to_id_mapping = {"pistol": 0,
                           "knife": 1}

# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov5(info_dict):
    print_buffer = []
    
    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    save_file_name = os.path.join("annotations2", info_dict["filename"].replace("JPG", "txt"))
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))


In [9]:
# convert all annotations with a loop - run once to create annotations

# # Get the annotations
# annotations = [os.path.join('annotations', x) for x in os.listdir('./data/Knife_detection/annotations') if x[-3:] == "xml"]
# # annotations = [os.path.join(x) for x in os.listdir('./data/Pistol detection/xmls') if x[-3:] == "xml"]
# annotations.sort()
# print(annotations)

# print(annotations)
# # Convert and save the annotations

# for ann in tqdm(annotations):
#     print(ann)
#     info_dict = extract_info_from_xml(os.path.join('./data/Knife_detection', ann))
#     convert_to_yolov5(info_dict)
# annotations = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "txt"]
# print(annotations)

In [10]:
# THIS IS TEMP AND ONE-TIME
# ##########################

# def add_numbers(filename):
#     with open(filename, 'r') as readfile:
#         data = readfile.readlines()
#     with open(filename, 'w') as writefile:
#         for i, line in enumerate(data):
#             writefile.write('%s %s' % (line, '.txt'))

# filename = './annotations/armas (1)'
# add_numbers(filename)

# for path, _, filenames in os.walk(folder):
#     for filename in filenames:
#         add_numbers(os.path.join(path, filename))





### USE THIS:
#############
# from pathlib import Path
# for file in range(3,3001): 
#   p = Path('./annotations/armas (' + str(file) + ')')
#   p.rename(p.with_suffix('.txt'))
#############




# THIS IS TEMP AND ONE-TIME
# ##########################

In [15]:

# annotations = [os.path.join('annotation', x) for x in os.listdir('annotations') if x[-3:] == "txt"]
!pwd
annotations = os.listdir('./annotations')
print(annotations)

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

print(class_id_to_name_mapping)

def plot_bounding_box(image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size
    
    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h 
    
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
    
    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)), width = 5, outline = 'red')
        
        plotted_image.text((x0, y0 - 10), class_id_to_name_mapping[(int(obj_cls))], fill = 'green')
    
    plt.imshow(np.array(image))
    plt.show()

# Get any random annotation file 
annotation_file = random.choice(annotations)
print(annotation_file)
annotation_file = os.path.join('./annotations', annotation_file)
print(annotation_file)

with open(annotation_file, "r") as file:
    annotation_list = file.read().split("\n")[:-1]
    print(annotation_list)
    annotation_list = [x.split(" ") for x in annotation_list]
    annotation_list = [[float(y) for y in x ] for x in annotation_list]

#Get the corresponding image file
image_file = annotation_file.replace("annotations", "Weapons").replace("txt", "jpg")
print(image_file)
assert os.path.exists(image_file)

#Load the image
image = Image.open(image_file)

#Plot the Bounding Box
plot_bounding_box(image, annotation_list)

/content/drive/MyDrive/Colab Notebooks/046211_Deep_Learning/DL_project
['train', 'test']
{0: 'pistol', 1: 'knife'}
train
./annotations/train


IsADirectoryError: ignored

# Data split and reorganize

In [ ]:
# Read images and annotations
Weapons = [os.path.join('images', x) for x in os.listdir('images')]
annotations = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "txt"]

Weapons.sort()
annotations.sort()

# Split the dataset into train-valid-test splits 
train_images, test_images, train_annotations, test_annotations = train_test_split(Weapons, annotations, test_size = 0.2, random_state = 1)
# val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

In [ ]:
!mkdir images/train images/test annotations/train annotations/test

In [ ]:
#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        print(f)
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, 'images/train')
move_files_to_folder(train_annotations, 'annotations/train/')
move_files_to_folder(test_images, 'images/test/')
move_files_to_folder(test_annotations, 'annotations/test/')

# Dataset class definition

In [ ]:
class Weapon_Dataset(Dataset): # TODO - use xml package to read data from images and match with xml files
    
    def __init__(self, image_path,csv_file, Transform=None):
        # self.image_path = image_path
        # self.csv = pd.read_csv(csv_file) 
        # self.transform = Transform
        
    def __len__(self): 
      return len(self.csv)
    
    def __getitem__(self, idx): 
      if torch.is_tensor(idx):
        idx = idx.tolist()
       
       # read image
       # read matching xml file
       
    #  image = Image.open(os.path.join(self.image_path, self.csv.iloc[idx,0]))
    #  label = self.csv.iloc[idx,1]
        #if self.image_path == '../input/cassava-leaf-disease-classification/train_images':
      return self.transform(image), label


In [ ]:
from torch.utils.data import Dataset, DataLoader


train_dataset = Cassava_Dataset(image_path,train_csv,basic_transform)
valid_set = Cassava_Dataset(image_path,test_csv_path,test_transform)


# train_dataset = torchvision.datasets.
# valid_dataset = torchvision.datasets.
# test_dataset = torchvision.datasets.

batch_size = 128

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_set,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=batch_size,shuffle=True)
